# MMSBR preprocess step 3

In [1]:
import os
import time
import pickle
import pandas as pd
import numpy as np

datasets_name = 'Cell_Phones_and_Accessories'
data_path = 'dict/' + datasets_name +'/session_data.csv'

price_num = '100'

data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID', 'priceLevel', 'category']]

# dict (sessionID:[itemID,itemID])
sess_all = {}
# dict (sessionID:[priceLevel, priceLevel])
price_all = {}



# dict (sessionID:[cate, cate])
cate_all = {}
# dict (sessionID:[brand, brand])
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    price = row['priceLevel']
    cate = row['category']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
        price_all[sess_id].append(price)
        cate_all[sess_id].append(cate)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)
        price_all[sess_id] = []
        price_all[sess_id].append(price)
        cate_all[sess_id] = []
        cate_all[sess_id].append(cate)

sess_total = data['sessionID'].max()
split_num = int(sess_total/10*9)

tra_sess = dict() # dict(session_id:[item_id,item_id])
tes_sess = dict() 
tra_price = dict()# dict(session_id:[price,price])
tes_price = dict()
tra_cate = dict()# dict(session_id:[cate,cate])
tes_cate = dict()
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    all_price = price_all[sess_temp]
    all_cate = cate_all[sess_temp]
    if len(all_seqs) < 2:
        continue
    if len(all_seqs) > 20:
        all_seqs = all_seqs[:20]
        all_price = all_price[:20]
        all_cate = all_cate[:20]
    if int(sess_temp) < split_num:
        tra_sess[sess_temp] = all_seqs
        tra_price[sess_temp] = all_price
        tra_cate[sess_temp] = all_cate
    else:
        tes_sess[sess_temp] = all_seqs
        tes_price[sess_temp] = all_price
        tes_cate[sess_temp] = all_cate

item_dict = {} #dict(old_itemID: new_itemID)
cate_dict = {} #dict(old_cate: new_cate)
price_dict = {} #dict(old_price: new_price)


# item_id: price_level, item_id: rating
item_price = {} #dict[new_itemID: priceLevel]
item_cate = {} #dict[new_itemID: cate]




# tra_sess tra_price tra_cate
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
#     global item_ctr
#     print(item_ctr)
    train_seqs = []
    train_price = []
    train_cate = []
#     train_brand = []
    item_ctr = 1
    price_ctr = 1
    cate_ctr = 1
    for s in tra_sess:
        seq = tra_sess[s]
        price_seq = tra_price[s]
        cate_seq = tra_cate[s]
        outseq = []
        pri_outseq = []
#         rating_outseq = []
        cate_outseq = []
        for (i, j_pri, w_cate) in zip(seq, price_seq, cate_seq):
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
            if j_pri in price_dict:
                pri_outseq += [price_dict[j_pri]]
            else:
                pri_outseq += [price_ctr]
                price_dict[j_pri] = price_ctr
                price_ctr += 1
            if w_cate in cate_dict:
                cate_outseq += [cate_dict[w_cate]]
            else:
                cate_outseq += [cate_ctr]
                cate_dict[w_cate] = cate_ctr
                cate_ctr += 1
            if item_dict[i] not in item_price:
                item_price[item_dict[i]] = price_dict[j_pri]
                item_cate[item_dict[i]] = cate_dict[w_cate]
        if len(outseq) < 2:  # Doesn't occur
            print('session length is 1')
            continue
        train_seqs += [outseq]
        train_price += [pri_outseq]
        train_cate += [cate_outseq]
    print("#train_session",len(train_seqs))
    print("#train_items",item_ctr-1)
    print("#train_price",price_ctr-1)
    print("#train_category",cate_ctr-1)
    return train_seqs, train_price, train_cate


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_seqs = []
    test_price = []
    test_cate = []
    for s in tes_sess:
        outseq = []
        out_price = []
        out_rating = []
        out_cate = []
        for i, j, w in zip(tes_sess[s], tes_price[s], tes_cate[s]):
            if i in item_dict:
                outseq += [item_dict[i]]
                out_price += [price_dict[j]]
                out_cate += [cate_dict[w]]
        if len(outseq) < 2:
            print('obtain test session length is 1')
            continue
        test_seqs += [outseq]
        test_price += [out_price]
        test_cate += [out_cate]
    print("#test_session: ",len(test_seqs))
    return test_seqs, test_price, test_cate

# Convert test sessions to sequences, renumbering items that do not appear in training set
def obtian_tes_cold():
    test_seqs = []
    test_price = []
    test_seqs_cold = []
    test_price_cold = []
    num_cold = 0
    global item_ctr,price_ctr
#     test_cate = []
    for s in tes_sess:
        outseq = []
        out_price = []
        contain_cold = False
        for i, p in zip(tes_sess[s], tes_price[s]):
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                num_cold += 1
                contain_cold = True
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
            if p in price_dict:
                out_price += [price_dict[p]]
            else:
                out_price += [price_ctr]
                price_dict[p]= price_ctr
                price_ctr += 1
#             if i in item_dict:
#                 outseq += [item_dict[i]]
#                 out_price += [price_dict[j]]

        if len(outseq) < 2:
            print('obtain test session length is 1')
            continue
        if contain_cold:
            test_seqs_cold += [outseq]
            test_price_cold += [out_price]
        else:
            test_seqs += [outseq]
            test_price += [out_price]
    print("#test_session: ",len(test_seqs))
    print("#cold_test_session: ",len(test_seqs_cold))
    print('#cold_items',str(num_cold))
    return test_seqs, test_price, test_seqs_cold, test_price_cold


# 数据增强
def process_seqs(iseqs, iprice, icate):
    out_seqs = []
    out_price = []
    out_cate = []
    labs = []
    max_length = 19
    for seq, pri, rat, cat in zip(iseqs, iprice, icate):
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
            out_price += [pri[:-i]]
            out_cate += [cat[:-i]]
    return out_seqs, out_price, out_cate, labs

# 无数据增强
def process_seqs_no(iseqs, iprice, icate):
    print("no data augment")
    out_seqs = []
    out_price = []
    out_cate = []
    labs = []
    max_length = 19
    for seq, pri, cat in zip(iseqs, iprice, icate):     
        labs += [seq[-1]]
        out_seqs += [seq[:-1]]
        out_price += [pri[:-1]]
        out_cate += [cat[:-1]]
    return out_seqs, out_price, out_cate, labs

tra_seqs, tra_pri, tra_cat  = obtian_tra()
tes_seqs, tes_pri, tes_cat  = obtian_tes()
# save item_dict:(old_itemID: new_itemID) nocold
old2new_nocold_path = './dict/' + datasets_name + '/old2newNocold.csv'
old_list_nocold = list(item_dict.keys())
new_list_nocold = list(item_dict.values())
old2new_nocold_dict = {'old':old_list_nocold, 'new':new_list_nocold}
data_old_new_no = pd.DataFrame(old2new_nocold_dict)
data_old_new_no.to_csv(old2new_nocold_path)

tr_seqs, tr_pri, tr_cat, tr_labs = process_seqs_no(tra_seqs, tra_pri, tra_cat)
te_seqs, te_pri, te_cat, te_labs = process_seqs_no(tes_seqs, tes_pri, tes_cat)

# cold_start
# te_seqs_nocold, te_pri_nocold, te_labs_nocold = process_seqs_no(tes_seqs_nocold, tes_pri_nocold)
# te_seqs_cold, te_pri_cold, te_labs_cold = process_seqs_no(tes_seqs_cold, tes_pri_cold)

print('train sequence: ',tr_seqs[:5])
print('train price: ',tr_pri[:5])
print('train category: ',tr_cat[:5])
print('train lab: ',tr_labs[:5])


# 增加 all_item_price_list & all_item_rating_list

ip_id_list = list(item_price.keys())
ip_price_list = list(item_price.values())
id_price_dict = {'id':ip_id_list, 'price':ip_price_list}
id_price_dict = pd.DataFrame(id_price_dict)
id_price_dict = id_price_dict.reset_index()[['id', 'price']]

id_price_dict.sort_values(by=["id"],inplace=True,ascending=[True])
price_list_save = id_price_dict['price'].tolist()

ic_id_list = list(item_cate.keys())
ic_cate_list = list(item_cate.values())
id_cate_dict = {'id':ic_id_list, 'cate':ic_cate_list}
id_cate_dict = pd.DataFrame(id_cate_dict)
id_cate_dict = id_cate_dict.reset_index()[['id', 'cate']]

id_cate_dict.sort_values(by=["id"],inplace=True,ascending=[True])
cate_list_save = id_cate_dict['cate'].tolist()



tra = (tr_seqs, tr_pri, tr_cat, price_list_save, cate_list_save, tr_labs)
tes = (te_seqs, te_pri, te_cat, price_list_save, cate_list_save, te_labs)

all_train = 0
all_tes = 0
for seq in tra_seqs:
    all_train += len(seq)
for seq in tes_seqs:
    all_tes += len(seq)
print('#train_interactions: ',all_train)
print('#test_interactions: ',all_tes)

print('#train_session: ',(len(tra_seqs)))
print('#test_session: ',(len(tes_seqs)))

print('sequence average length: ', (all_train+all_tes)/(len(tra_seqs) + len(tes_seqs) * 1.0))
train_data_path = './datasets/' + datasets_name + str(price_num) + 'logistic'

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_normal_test = train_data_path + "/test.txt"

pickle.dump(tra, open(path_data_train, 'wb'))
pickle.dump(tes, open(path_normal_test, 'wb'))


# get asin item_id 获得asin list 有顺序, 给处理 img及text
asin2old_path = 'dict/' + datasets_name +'/asin2itemID.csv'
old2new_path = 'dict/' + datasets_name +'/old2newNocold.csv'



asin2old = pd.read_csv(asin2old_path)
asin2old = asin2old.rename(columns={ 'itemID':'old'})
asin2old = asin2old[['asin', 'old']]


old2new = pd.read_csv(old2new_path)
old2new = old2new[['old', 'new']]


asin2new = pd.merge(asin2old, old2new, how='left', on = 'old')


asin2new = asin2new.dropna(axis=0)
asin2ItemID = asin2new.reset_index()[['asin', 'new']]

asin2ItemID.sort_values(by=["new"],inplace=True,ascending=[True])


asin_list = asin2ItemID['asin'].tolist()
# new_list = asin2ItemID['new'].tolist()
asin_list_save = train_data_path + '/asinlist.npy'
np.save(asin_list_save, asin_list)



print("dataset: ", datasets_name)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("data formulation: id_seq, price_seq, cate_seq, price_list, cate_list, labs")
print("done")



#train_session 70260
#train_items 8614
#train_price 10
#train_category 48
#test_session:  7766
no data augment
no data augment
train sequence:  [[1], [3], [5], [7, 8], [10, 11]]
train price:  [[1], [3], [3], [4, 1], [3, 1]]
train category:  [[1], [2], [3], [5, 5], [6, 2]]
train lab:  [2, 4, 6, 9, 12]
#train_interactions:  176784
#test_interactions:  19592
#train_session:  70260
#test_session:  7766
sequence average length:  2.5168020916104887
dataset:  Cell_Phones_and_Accessories
2022-09-23 16:34:44
data formulation: id_seq, price_seq, cate_seq, price_list, cate_list, labs
done
